# Spring and Mass

We want to know the solution to the problem: $ F = m\cdot a$, making $a = -c/m\cdot v -k/m\cdot x + f(t)$, leading to:

$\ddot{x} = - \frac{c}{m}\cdot\dot{x} - \frac{k}{m} \cdot x + \frac{f(t)}{m}$

The state space representation of this problem is:
$$\underbrace{\begin{bmatrix}\ddot{x}\\
\dot{x}\end{bmatrix}}_{\dot{w}} = 
\underbrace{\begin{bmatrix} -c/m & -k/m \\
1 & 0\end{bmatrix}}_{A} \cdot \underbrace{\begin{bmatrix}\dot{x}\\
x\end{bmatrix}}_{w}+\underbrace{\begin{bmatrix}f(t)\\
0\end{bmatrix}}_{B}$$

We can define a function that outputs the change in $x$ and $\dot{x}$ as:

In [ ]:
import numpy as np
import math

In [ ]:
def spring(t, w, f= lambda t,w: 0, k=1E5, c=2E3, m=450):
    ''' Linear model of spring-mass-damper system

    Args:
    t: time (s)
    w: state vector [displacement (m), velocity (m/s)]
    f: external force function (N)
    k: spring constant (N/m)
    c: damping coefficient (N·s/m)
    m: mass (kg)
    '''

    # unpack state vector
    assert(np.array(w).size == 2)
    x, v, = w

    # compute derivatives
    dv = (-k*x - c*v + f(t,w))/m # Acceleration
    dx = v # Velocity

    return np.array([dx, dv])

In [ ]:
def solve(f, t_span, w_0:np.array, f_ext = lambda t,w:0, n_steps=1000,controller=None):
    ## Initialize Solution Array
    t_sol = np.linspace(*t_span, n_steps)
    w_sol_T = np.ndarray((t_sol.size,w_0.size))
    w_sol_T[0] = w_0

    ## Solve
    for i in range(t_sol.size-1):
        if controller: controller.dt = t_sol[1]-t_sol[0]
        f_cont = (lambda t,w: 0) if not controller else controller
        f_tot = lambda t,w: f_ext(t,w)+f_cont(t,w)
        dw = f(t_sol[i],w_sol_T[i],f_tot)
        dt = t_sol[i+1]-t_sol[i]
        w_sol_T[i+1] = w_sol_T[i] + dw*dt
    w_sol = w_sol_T.T
    return t_sol, w_sol

In [ ]:
def plot_sol(t_sol:np.array, w_sol:np.ndarray, labels:list=None, title:str=None, legend:str=None, plot:list=None):
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    n_vars = w_sol.shape[0]

    if isinstance(plot,list):
        assert(len(plot)==n_vars)
    else:
        plot = [1 for i in range(n_vars)]
    plot = np.bool(plot)
    n_plots = int(sum(plot))

    # Treat Labels
    labels_def = [f'x{i+1}' for i in range(n_vars)]
    if isinstance(labels,list):
        labels = [labels[i] if len(labels)>i else labels_def[i] for i in range(n_vars)]
    else:
        labels = labels_def
    
    # Make Figure and Subplots
    fig = make_subplots(rows=n_plots, cols=1)
    legend = legend if legend else 'Sol 1'

    for i in range(n_vars):
        if plot[i]==1:
            plotnum = int(sum(plot[:i+1]))
            fig.add_trace(go.Scatter(x=t_sol, y=w_sol[i], name=legend, legendgroup=plotnum, legendgrouptitle_text=labels[i]), row=plotnum, col=1)
            fig.update_xaxes(dict(showticklabels=False), row=plotnum, col=1)
            fig.update_yaxes(dict(title_text=labels[i]), row=plotnum, col=1)
    fig.update_xaxes(dict(title_text="Time",showticklabels=True), row=n_plots, col=1)
    fig.update_layout(dict(title=title, legend_tracegroupgap = 200/n_plots))
    return fig

def add_sol(fig, t_sol, w_sol, legend:str=None, plot:list=None):
    import plotly.graph_objects as go

    n_vars = w_sol.shape[0]

    if isinstance(plot,list):
        assert(len(plot)==n_vars)
    else:
        plot = [1 for i in range(n_vars)]
    plot = np.bool(plot)
    n_plots = int(sum(plot))

    assert(n_plots == len(set([fig.to_dict()['data'][i]['legendgroup'] for i in range(len(fig.to_dict()['data']))])))
    n_sols = len(fig.to_dict()['data'])/n_plots
    legend = legend if legend else f'Sol {n_sols+1:g}'
    for i in range(n_vars):
        if plot[i]==1:
            plotnum = int(sum(plot[:i+1]))
            fig.add_trace(go.Scatter(x=t_sol, y=w_sol[i], name=legend, legendgroup=plotnum), row=plotnum, col=1)
    return fig

In [ ]:
# Solve and Plot Spring and Damper
# Initial Conditions
w_0 = np.array([0, 0.0])

# Parameters
t_span = [0,5]
steps = 1000
f_ext = lambda t,w: 0 if t<0.5 or t>2.5 else -200*9.81

k_vals = [1e5,1e5,1e5]
c_vals = [2e3,5e3,1e4]

# Solve and Plot
fig = None
for i in range(len(k_vals)):
    k = k_vals[i]
    c = c_vals[i]
    legend=f'Sol {i+1} (k={k:g}, c={c:g})'
    if not fig: fig = plot_sol(*solve(lambda t,w,f: spring(t,w,f,c=c, k=k), t_span, w_0,f_ext, n_steps=1000),["x[m]","v[m/s]"], legend=legend,
                title = "Simulation of Spring and Damper System with Weight added on t=0.5 and removed in t=2.5", plot=[1,0])
    else: add_sol(fig, *solve(lambda t,w,f: spring(t,w,f,c=c, k=k), t_span, w_0,f_ext), legend=legend, plot=[1,0])
fig.show()

# Box, Train and Rail System

In the box, train and Rail System we have two different elements (the falling box and the train) whose x and y position we can observe, and want to model the effect of actuators on the train that would make the difference between $x_{\text{box}}$ and $x_{\text{train}}$ 0 at the moment of landing.

For the box, $\dot{x} = 0$ and $\ddot{y} = -g$.

The train is located over an inclined rail which inflicts friction, such that there are three forces acting on the $x$-axis:

$F_{act} → \text{External Force from the Actuator}\\
F_{gx} → \text{Gravity Component on the x-axis}\\
F_{f} → \text{Friction from Contact with the Rail}$

Assuming that the actuator can produce diagonal forces, the function that governs movement on $x$ is then: $F_x=m\cdot a_x$, which leads to:

$a_x=-g\cdot \text{sin}(\theta) -\mu\cdot g\cdot \text{cos}(\theta)\cdot \text{sign}(\dot{x}) + \frac{\text{cos}(\theta)}{m}\cdot f(t)\\
a_y = -g\cdot \text{cos}(\theta) + \frac{\text{sin}(\theta)}{m}\cdot f(t)$

This is not a linear system, and we will not attempt to represent it as a state-space equation.

In [ ]:
def train_and_rail(t, w, f=lambda t,w: 0, m=100, g=9.81, theta=math.pi/6, mu=1e-4):
    def sign(x):
        return 0 if x==0 else (1 if x>0 else -1)

    x_box, y_box, vy_box, x_train, vx_train, y_train = w #unpack state vector

    free_fall = not((y_box-y_train)<=1 and abs(x_box-x_train)<1)

    dx_box = 0
    dy_box = vy_box if free_fall else 0
    dvy_box = -g if free_fall else 0
    dx_train = vx_train
    dvx_train = -g*math.sin(theta) - mu*g*math.cos(theta)*sign(vx_train) + math.cos(theta)*f(t,w)
    dy_train = vx_train/math.cos(theta)*math.sin(theta)

    return np.array([dx_box, dy_box, dvy_box, dx_train, dvx_train, dy_train])

In [ ]:
class pid:
    def __init__(self, K_p, K_d, K_i, error=lambda t, w:0):
        self.dt = None
        self.K_p = K_p
        self.K_d = K_d
        self.K_i = K_i

        self.f_a = []
        self.e = []
        self.e_dot = []
        self.e_int = []

        self.error = error
    
    def __call__(self, t, w):
        e = self.error(t,w)
        self.e.append(e)
        if len(self.e)==1:
            self.e_dot.append(0)
            self.e_int.append(0)
        else:
            self.e_dot.append((self.e[-1]-self.e[-2])/self.dt)
            self.e_int.append(self.e_int[-1]+(self.e[-2]+self.e[-1])/2*self.dt)
        action = self.K_p*self.e[-1]+self.K_d*self.e_dot[-1]+self.K_i*self.e_int[-1]
        self.f_a.append(action)
        return action

In [ ]:
def plot_animation(t_sol, w_sol, controller, SKIPFRAMES=2, theta=math.pi/6):
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    rows = 4
    cols = 3
    x_max = 120
    fig = make_subplots(rows=rows, cols=cols,
                        specs=[[{"rowspan":3, "colspan":2}, None, {}],
                            [None, None, {}],
                            [None, None, {}],
                            [{}, {}, {}]],
                        subplot_titles=("PID Train Simulation", "x [m]", "v [m/s]", "a [m/s²]", "ε(x)", "Δε(x)", "∫Δε(x)"))

    x_box_sol, y_box_sol, vy_box_sol, x_train_sol, vx_train_sol, y_train_sol = w_sol
    a_sol = [0] + np.diff(vx_train_sol/t_sol[1],1).tolist()
    # Precompute frames
    len_t = len(t_sol)
    # Only create frames for sampled timesteps to speed up animation
    sampled_idx = list(range(0, len_t,SKIPFRAMES))
    frame_amount = len(sampled_idx)
    frames = []
    frame_meta = []
    for num in range(frame_amount):
        idx_t = sampled_idx[num]

        platform_x = [x_train_sol[idx_t]-3.1, x_train_sol[idx_t]+3.1]
        platform_y = [y_train_sol[idx_t], y_train_sol[idx_t]]
        cube_x = [x_box_sol[idx_t]-1, x_box_sol[idx_t]+1]
        cube_y = [y_box_sol[idx_t], y_box_sol[idx_t]]

        # traces for main (row1 col1)
        main_traces = [
            go.Scatter(x=[0, x_max], y=[5, x_max*np.tan(theta)+5], mode='lines', line=dict(color='black', width=6)),
            go.Scatter(x=platform_x, y=platform_y, mode='lines', line=dict(color='blue', width=18)),
            go.Scatter(x=cube_x, y=cube_y, mode='lines', line=dict(color='black', width=14)),
        ]

        # Add a time annotation (as a text trace) positioned in the main subplot
        time_text = go.Scatter(x=[x_max*0.05], y=[x_max*0.9],
                            mode='text', text=[f"t={t_sol[idx_t]:.2f}s"], textfont=dict(size=16, color='black'), textposition="bottom right")

        # time series traces (use partial data up to idx_t)
        ts_traces = [
            go.Scatter(x=t_sol[:idx_t+1], y=x_train_sol[:idx_t+1], mode='lines', line=dict(color='blue'), name='x'),
            go.Scatter(x=t_sol[:idx_t+1], y=vx_train_sol[:idx_t+1], mode='lines', line=dict(color='blue'), name='v'),
            go.Scatter(x=t_sol[:idx_t+1], y=a_sol[:idx_t+1], mode='lines', line=dict(color='blue'), name='a'),
            go.Scatter(x=t_sol[:idx_t+1], y=controller.e[:idx_t+1], mode='lines', line=dict(color='blue'), name='ε(x)'),
            go.Scatter(x=t_sol[:idx_t+1], y=controller.e_dot[:idx_t+1], mode='lines', line=dict(color='blue'), name='Δε(x)'),
            go.Scatter(x=t_sol[:idx_t+1], y=controller.e_int[:idx_t+1], mode='lines', line=dict(color='blue'), name='∫Δε(x)'),
        ]

        # Build frame with updates for each subplot index
        frame = go.Frame(data=main_traces + [time_text] + ts_traces, name=str(num))
        frames.append(frame)

    # Initial static traces (frame 0)
    init_main = frames[0].data[:4]  # now includes time_text and trial_text as 4th  in main group
    init_ts = frames[0].data[4:]

    # Place main traces into (1,1) (spans rows 1-3 cols 1-2)
    for tr in init_main:
        fig.add_trace(tr, row=1, col=1)

    # Place time series traces into their subplots
    fig.add_trace(init_ts[0], row=1, col=3)  # displ
    fig.add_trace(init_ts[1], row=2, col=3)  # vel
    fig.add_trace(init_ts[2], row=3, col=3)  # accel
    fig.add_trace(init_ts[3], row=4, col=1)  # error
    fig.add_trace(init_ts[4], row=4, col=2)  # e_dot
    fig.add_trace(init_ts[5], row=4, col=3)  # e_int

    # Layout adjustments: fix axis ranges so they don't autoscale per frame
    fig.update_xaxes(range=[0, x_max], row=1, col=1)
    fig.update_yaxes(range=[0, x_max], row=1, col=1)

    # Time-series axes ranges (fix across all relevant subplots)
    fig.update_xaxes(range=t_span, row=1, col=3)
    fig.update_xaxes(range=t_span, row=2, col=3)
    fig.update_xaxes(range=t_span, row=3, col=3)
    fig.update_xaxes(range=t_span, row=4, col=1)
    fig.update_xaxes(range=t_span, row=4, col=2)
    fig.update_xaxes(range=t_span, row=4, col=3)

    # Fix y-limits for time series to the min/max computed once so they don't change per frame
    def fixed_ylim(arr):
        vmin = np.min(arr)
        vmax = np.max(arr)
        padding = max(abs(vmin), abs(vmax)) * 0.1 if max(abs(vmin), abs(vmax))>0 else 1
        return vmin - padding, vmax + padding

    displ_ylim = fixed_ylim(x_train_sol)
    vrail_ylim = fixed_ylim(vx_train_sol)
    a_rail_ylim = fixed_ylim(a_sol)
    e_ylim = fixed_ylim(controller.e)
    edot_ylim = fixed_ylim(controller.e_dot)
    eint_ylim = fixed_ylim(controller.e_int)

    fig.update_yaxes(range=displ_ylim, row=1, col=3)
    fig.update_yaxes(range=vrail_ylim, row=2, col=3)
    fig.update_yaxes(range=a_rail_ylim, row=3, col=3)
    fig.update_yaxes(range=e_ylim, row=4, col=1)
    fig.update_yaxes(range=edot_ylim, row=4, col=2)
    fig.update_yaxes(range=eint_ylim, row=4, col=3)

    fig.frames = frames
    fig.update_layout(showlegend=False, height=600, width=1000,
                    title_text=None)

    # Animation settings — use FRAME_DURATION_MS and avoid full redraw when possible
    fig.update_layout(updatemenus=[dict(type='buttons', showactive=True,
                        y=1.05, x=1.10,
                        xanchor='right', yanchor='top',
                        buttons=[
                {"args": [None, {"frame": {"duration": 0, "redraw": False},
                                    "fromcurrent": True,
                                    "transition": {"duration": 0,"easing": "quadratic-in-out"}}],
                                    "label": "Play/Pause","method": "animate",
                "args2": [[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}],
                                    "label": "Play/Pause", "method": "animate"}
                                ]
                            )])

    # Disable global layout transition easing to avoid slow implicit transitions when frames change
    fig.update_layout(transition={"duration": 1, "easing": "linear"})

    # Build an explicit slider with immediate-mode steps and zero-duration transitions so
    # clicking/dragging the slider updates frames instantly (no implicit tweening).
    steps = []
    for num in range(len(frames)):
        idx_t = sampled_idx[num]        
        # show label only when time is exactly 0 (trial start) or a multiple of 0.5s
        label = f"{t_sol[idx_t]:.2f}s"
        step = dict(
            value=num,
            method='animate',
            label=label,
            visible=True,
            args=[[frames[num].name], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}]
        )
        steps.append(step)

    # Use a fixed small top padding so the slider doesn't push the plotting area down when many steps exist
    slider = dict(
        active=0,
        currentvalue={"font": {"size": 14, "weight": 20, "color": "black"}},
        pad={"t": 0},
        steps=steps,
        x=0,
        xanchor='left',
        y=-0.05,
        yanchor='top',
        ticklen=0,
        minorticklen=0,
        font=dict(size=1,weight=1, color='white')
        )

    fig.update_layout(sliders=[slider])
    return fig

In [ ]:
controller = pid(100,10,1,error = lambda t, w: w[0]-w[3])
theta= math.pi/6
x_box_0 = np.random.random()*120
y_box_0 = 100
x_train_0 = 0
y_train_0 = 5+math.sin(theta)*x_train_0
w_0 = np.array([x_box_0,y_box_0,0,x_train_0,0,y_train_0])
f = lambda t,w: 0
t_sol, w_sol = solve(lambda t,w,f,:train_and_rail(t,w,f),[0,5],w_0, controller=controller, n_steps=250)

plot_animation(t_sol, w_sol, controller, SKIPFRAMES=5).show()
#plot_sol(t_sol, w_sol,labels=['x_box', 'y_box', 'vy_box', 'x_train', 'vx_train', 'y_train'], plot=[0,1,0,1,0,1]).show()
